In [ ]:
from google.colab import drive

drive.mount('/content/gdrive') 

In [ ]:
!git clone https://github.com/kashperova/ssl-hsi-course-work.git 

In [ ]:
%cd ssl-hsi-course-work/src 

In [1]:
from models.ca_ffdn import CA_FFDN
from modules.trainers.supervised import BaseSupervisedTrainer
from modules.datasets.hsi import HyperspectralDataset
from config.train_config import BaseTrainConfig
from utils.seed import set_seed
from utils.metrics import Metrics, Task
from utils.data import load_hsi_dataset

import os
import wandb
import torch.optim as optim

from torch import nn

In [2]:
os.environ["WANDB_API_KEY"] = ""

In [10]:
wandb.init()

wandb: Currently logged in as: skashperova (kashperova-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
set_seed(42)

Random seed set to 42


In [4]:
class TrainConfig(BaseTrainConfig):
    epochs: int = 100
    train_batch_size: int = 64
    eval_batch_size: int = 64
    train_test_split: float = 0.7

In [5]:
patches, labels = load_hsi_dataset(
    dataset_name="IndianPines", root_dir="../data", pca_components=30, patch_size=15
)

In [6]:
dataset = HyperspectralDataset(patches, labels)

In [8]:
model = CA_FFDN(in_channels=30, num_classes=16)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.5)
criterion = nn.CrossEntropyLoss()

trainer = BaseSupervisedTrainer(
    model=model,
    optimizer=optimizer,
    lr_scheduler=scheduler,
    dataset=dataset,
    config=TrainConfig(),
    metrics=Metrics(task=Task.MULTICLASS_CLASSIFICATION, num_classes=16, average="micro"),
    loss_fn=criterion
)

In [9]:
trainer.train(verbose=True)

Training:   0%|          | 0/100 [00:47<?, ?it/s]


Error: You must call wandb.init() before wandb.log()